In [1]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_openai import OpenAI
from langchain_pinecone import PineconeVectorStore


/opt/miniconda3/envs/medchat/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


KeyboardInterrupt: 

In [2]:
%pwd

'/Users/ahir/disease_assitance_service/research'

In [ ]:
changedir ../disease_assitance_service/researc

SyntaxError: invalid syntax (508405475.py, line 1)

In [1]:
from langchain_community.document_loaders import PyPDFLoader
from tqdm import tqdm

loader = PyPDFLoader("book.pdf")
documents = []

for i in tqdm(range(loader._get_num_pages()), desc="Loading pages"):
    try:
        page = loader.load_page(i)
        documents.append(page)
    except Exception as e:
        print(f"⚠️ Skipped page {i}: {e}")

print(f"✅ Successfully loaded {len(documents)} pages.")


/opt/miniconda3/envs/medchat/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ValueError: File path book.pdf is not a valid file or url

In [2]:
%pwd


'/Users/ahir/disease_assitance_service/research'

In [4]:
from langchain_community.document_loaders import PyPDFLoader

# Correct relative path (go up one folder, then into data/)
loader = PyPDFLoader("../data/book.pdf")
docs = loader.load()

print(f"✅ Loaded {len(docs)} pages successfully!")
print(docs[0].page_content[:500])  # preview first few lines of text


✅ Loaded 4505 pages successfully!



In [7]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from tqdm import tqdm

# Initialize text splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
    separators=["\n\n", "\n", " ", ""]
)

# Split your PDF pages into text chunks
texts = text_splitter.split_documents(docs)

print(f"✅ Split {len(docs)} pages into {len(texts)} text chunks.")
print(texts[0].page_content[:500])


✅ Split 4505 pages into 23444 text chunks.
The GALE
ENCYCLOPEDIA of
MEDICINE
THIRD EDITION


In [9]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from tqdm import tqdm

# Initialize embedding model
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Generate embeddings for all chunks
print("🔍 Creating embeddings for all text chunks, please wait...")

# This may take time depending on how many chunks you have
vectors = [embedding_model.embed_query(chunk.page_content) for chunk in tqdm(texts)]

print(f"✅ Created embeddings for {len(vectors)} chunks.")
print(f"Each embedding vector has {len(vectors[0])} dimensions.")


🔍 Creating embeddings for all text chunks, please wait...


100%|██████████| 23444/23444 [06:06<00:00, 64.04it/s]

✅ Created embeddings for 23444 chunks.
Each embedding vector has 384 dimensions.


In [ ]:
from langchain_community.vectorstores import FAISS

# Create and store FAISS vector index locally
print("💾 Building FAISS vector store...")

vector_store = FAISS.from_texts(
    [t.page_content for t in texts],
    embedding_model
)

# Save for reuse
vector_store.save_local("vector_store")

print("✅ Vector store created and saved in ./vector_store/")


💾 Building FAISS vector store...


ImportError: Could not import faiss python package. Please install it with `pip install faiss-gpu` (for CUDA supported GPU) or `pip install faiss-cpu` (depending on Python version).

In [12]:
from langchain_community.vectorstores import FAISS

# Create and store FAISS vector index locally
print("💾 Building FAISS vector store...")

vector_store = FAISS.from_texts(
    [t.page_content for t in texts],
    embedding_model
)

# Save for reuse
vector_store.save_local("vector_store")

print("✅ Vector store created and saved in ./vector_store/")


💾 Building FAISS vector store...


ValueError: input not a numpy array

In [13]:
import numpy as np
from langchain_community.vectorstores import FAISS

print("💾 Building FAISS vector store...")

# Convert your embeddings list into a NumPy array
emb_array = np.array(vectors, dtype="float32")

# Create FAISS index directly
vector_store = FAISS.from_embeddings(
    texts=[t.page_content for t in texts],
    embeddings=emb_array,
    embedding=embedding_model
)

# Save index
vector_store.save_local("vector_store")

print("✅ Vector store created and saved in ./vector_store/")


💾 Building FAISS vector store...


TypeError: FAISS.from_embeddings() missing 1 required positional argument: 'text_embeddings'

In [14]:
vector_store = FAISS.from_texts(
    [t.page_content for t in texts],
    embedding_model
)



KeyboardInterrupt: 

In [15]:
import numpy as np
from langchain_community.vectorstores import FAISS

print("💾 Building FAISS vector store...")

emb_array = np.array(vectors, dtype="float32")

vector_store = FAISS.from_embeddings(
    texts=[t.page_content for t in texts],
    embeddings=emb_array,
    embedding=embedding_model
)

vector_store.save_local("vector_store")
print("✅ Vector store created and saved in ./vector_store/")



💾 Building FAISS vector store...


TypeError: FAISS.from_embeddings() missing 1 required positional argument: 'text_embeddings'

In [16]:
import numpy as np
from langchain_community.vectorstores import FAISS

print("💾 Building FAISS vector store...")

# Convert list of embeddings to numpy array
emb_array = np.array(vectors, dtype="float32")

# Pair embeddings with text
text_embeddings = list(zip([t.page_content for t in texts], emb_array))

# Create FAISS index directly from embeddings
vector_store = FAISS.from_embeddings(
    text_embeddings=text_embeddings,
    embedding=embedding_model
)

# Save it locally
vector_store.save_local("vector_store")

print("✅ Vector store created and saved in ./vector_store/")


💾 Building FAISS vector store...


ValueError: input not a numpy array

In [18]:
import numpy as np
from langchain_community.vectorstores import FAISS

print("💾 Building FAISS vector store...")

# Make sure all embeddings are consistent and properly shaped
emb_array = np.vstack([np.array(v, dtype=np.float32) for v in vectors])

# Pair embeddings with their corresponding text chunks
text_embeddings = list(zip([t.page_content for t in texts], emb_array))

# Build FAISS index
vector_store = FAISS.from_embeddings(
    text_embeddings=text_embeddings,
    embedding=embedding_model
)

# Save FAISS index locally
vector_store.save_local("vector_store")

print("✅ Vector store created and saved in ./vector_store/")


💾 Building FAISS vector store...


ValueError: input not a numpy array

In [19]:
import numpy as np
from langchain_community.vectorstores import FAISS

print("💾 Building FAISS vector store...")

# Make sure all embeddings are consistent and properly shaped
emb_array = np.vstack([np.array(v, dtype=np.float32) for v in vectors])

# Pair embeddings with their corresponding text chunks
text_embeddings = list(zip([t.page_content for t in texts], emb_array))

# Build FAISS index
vector_store = FAISS.from_embeddings(
    text_embeddings=text_embeddings,
    embedding=embedding_model
)

# Save FAISS index locally
vector_store.save_local("vector_store")

print("✅ Vector store created and saved in ./vector_store/")


💾 Building FAISS vector store...


ValueError: input not a numpy array

In [20]:
import faiss
import numpy as np
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_core.documents import Document
import uuid

print("💾 Building FAISS vector store manually...")

# Convert your embedding vectors to a NumPy array
emb_array = np.vstack([np.array(v, dtype=np.float32) for v in vectors])

# Prepare FAISS index
embedding_dim = emb_array.shape[1]
index = faiss.IndexFlatL2(embedding_dim)

# Create mapping: ID -> Document
documents = [Document(page_content=t.page_content) for t in texts]
index_to_docstore_id = [str(uuid.uuid4()) for _ in range(len(documents))]
docstore = InMemoryDocstore(dict(zip(index_to_docstore_id, documents)))

# Build LangChain FAISS vectorstore manually
vector_store = FAISS(
    embedding_function=embedding_model,
    index=index,
    docstore=docstore,
    index_to_docstore_id=index_to_docstore_id,
)

# Add all vectors
vector_store.index.add(emb_array)

# Save locally
vector_store.save_local("vector_store")

print("✅ Vector store created and saved successfully in ./vector_store/")


💾 Building FAISS vector store manually...


ValueError: input not a numpy array

In [21]:
import faiss
import numpy as np
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_core.documents import Document
import uuid

print("🧹 Cleaning and validating embedding data...")

# Step 1: Ensure every vector is a flat list of floats with consistent size
clean_vectors = []
for i, v in enumerate(vectors):
    arr = np.array(v, dtype=np.float32).flatten()
    if arr.ndim == 1 and arr.size > 0:
        clean_vectors.append(arr)
    else:
        print(f"⚠️ Skipped invalid vector at index {i}")

# Step 2: Stack into a 2D array
emb_array = np.stack(clean_vectors)
print("✅ Embedding array shape:", emb_array.shape)

# Step 3: Create FAISS index
embedding_dim = emb_array.shape[1]
index = faiss.IndexFlatL2(embedding_dim)

# Step 4: Prepare documents and docstore
documents = [Document(page_content=t.page_content) for t in texts[:len(emb_array)]]
index_to_docstore_id = [str(uuid.uuid4()) for _ in range(len(documents))]
docstore = InMemoryDocstore(dict(zip(index_to_docstore_id, documents)))

# Step 5: Build vector store manually
vector_store = FAISS(
    embedding_function=embedding_model,
    index=index,
    docstore=docstore,
    index_to_docstore_id=index_to_docstore_id,
)

# Step 6: Add embeddings safely
print("💾 Adding embeddings to FAISS index...")
vector_store.index.add(np.ascontiguousarray(emb_array, dtype=np.float32))

# Step 7: Save locally
vector_store.save_local("vector_store")
print("✅ Vector store created and saved successfully in ./vector_store/")


🧹 Cleaning and validating embedding data...
✅ Embedding array shape: (23444, 384)
💾 Adding embeddings to FAISS index...


ValueError: input not a numpy array

In [22]:
import faiss
import numpy as np
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_core.documents import Document
import uuid

print("🧩 Building FAISS index (Mac-safe mode)...")

# Step 1: Ensure embeddings are contiguous float32 array
emb_array = np.ascontiguousarray(np.stack(vectors).astype("float32"))
print("✅ Embedding array shape:", emb_array.shape, "| dtype:", emb_array.dtype)

# Step 2: Initialize FAISS index
embedding_dim = emb_array.shape[1]
index = faiss.IndexFlatL2(int(embedding_dim))

# Step 3: Add embeddings directly to index
faiss.normalize_L2(emb_array)  # optional but improves retrieval
index.add(emb_array)
print("✅ Added", index.ntotal, "embeddings to FAISS index")

# Step 4: Build LangChain FAISS store manually
documents = [Document(page_content=t.page_content) for t in texts[:len(emb_array)]]
index_to_docstore_id = [str(uuid.uuid4()) for _ in range(len(documents))]
docstore = InMemoryDocstore(dict(zip(index_to_docstore_id, documents)))

vector_store = FAISS(
    embedding_function=embedding_model,
    index=index,
    docstore=docstore,
    index_to_docstore_id=index_to_docstore_id,
)

# Step 5: Save locally
vector_store.save_local("vector_store")
print("✅ Vector store created and saved successfully in ./vector_store/")


🧩 Building FAISS index (Mac-safe mode)...
✅ Embedding array shape: (23444, 384) | dtype: float32


ValueError: input not a numpy array

In [23]:
import faiss
import numpy as np
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_core.documents import Document
import uuid

print("🧩 Building FAISS index (safe fallback)...")

# Step 1: Ensure embeddings are contiguous float32 array
emb_array = np.ascontiguousarray(np.stack(vectors).astype("float32"))
print("✅ Embedding array shape:", emb_array.shape, "| dtype:", emb_array.dtype)

# Step 2: Initialize FAISS index
embedding_dim = emb_array.shape[1]
index = faiss.IndexFlatL2(int(embedding_dim))

# Step 3: Add embeddings directly (skip normalization)
index.add(np.ascontiguousarray(emb_array, dtype=np.float32))
print("✅ Added", index.ntotal, "embeddings to FAISS index")

# Step 4: Build LangChain FAISS store manually
documents = [Document(page_content=t.page_content) for t in texts[:len(emb_array)]]
index_to_docstore_id = [str(uuid.uuid4()) for _ in range(len(documents))]
docstore = InMemoryDocstore(dict(zip(index_to_docstore_id, documents)))

vector_store = FAISS(
    embedding_function=embedding_model,
    index=index,
    docstore=docstore,
    index_to_docstore_id=index_to_docstore_id,
)

# Step 5: Save locally
vector_store.save_local("vector_store")
print("✅ Vector store created and saved successfully in ./vector_store/")


🧩 Building FAISS index (safe fallback)...
✅ Embedding array shape: (23444, 384) | dtype: float32


ValueError: input not a numpy array

In [1]:
import faiss
import numpy as np
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_core.documents import Document
import uuid

print("🧩 Building FAISS index (Apple Silicon Safe Mode)...")

# Step 1: Create contiguous float32 array safely
emb_array = np.array(vectors, dtype=np.float32)
emb_array = np.require(emb_array, dtype=np.float32, requirements=["C_CONTIGUOUS", "ALIGNED"])
print("✅ Embedding array shape:", emb_array.shape, "| C_CONTIGUOUS:", emb_array.flags['C_CONTIGUOUS'])

# Step 2: Initialize FAISS index
embedding_dim = emb_array.shape[1]
index = faiss.IndexFlatL2(int(embedding_dim))

# Step 3: Add embeddings directly
index.add(emb_array)
print("✅ Added", index.ntotal, "embeddings to FAISS index")

# Step 4: Build LangChain FAISS wrapper
documents = [Document(page_content=t.page_content) for t in texts[:len(emb_array)]]
index_to_docstore_id = [str(uuid.uuid4()) for _ in range(len(documents))]
docstore = InMemoryDocstore(dict(zip(index_to_docstore_id, documents)))

vector_store = FAISS(
    embedding_function=embedding_model,
    index=index,
    docstore=docstore,
    index_to_docstore_id=index_to_docstore_id,
)

# Step 5: Save locally
vector_store.save_local("vector_store")
print("✅ Vector store created and saved successfully in ./vector_store/")


🧩 Building FAISS index (Apple Silicon Safe Mode)...


NameError: name 'vectors' is not defined

In [2]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("book.pdf")
docs = loader.load()
print(f"✅ Loaded {len(docs)} pages from book.pdf")


/opt/miniconda3/envs/medchat/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ValueError: All ufuncs must have type `numpy.ufunc`. Received (<ufunc 'sph_legendre_p'>, <ufunc 'sph_legendre_p'>, <ufunc 'sph_legendre_p'>)

In [3]:
import numpy as np
from langchain_community.document_loaders import PyPDFLoader

print("✅ NumPy version:", np.__version__)
print("✅ Import successful!")

loader = PyPDFLoader("book.pdf")
docs = loader.load()
print(f"✅ Loaded {len(docs)} pages successfully.")


ValueError: All ufuncs must have type `numpy.ufunc`. Received (<ufunc 'sph_legendre_p'>, <ufunc 'sph_legendre_p'>, <ufunc 'sph_legendre_p'>)

In [4]:
%pwd

'/Users/ahir/disease_assitance_service/research'

In [6]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("../data/book.pdf")
docs = loader.load()
print(f"✅ Loaded {len(docs)} pages successfully!")


ValueError: All ufuncs must have type `numpy.ufunc`. Received (<ufunc 'sph_legendre_p'>, <ufunc 'sph_legendre_p'>, <ufunc 'sph_legendre_p'>)

In [1]:
!clear

In [11]:
import faiss
print("✅ FAISS version:", faiss.__version__)


✅ FAISS version: 1.9.0
